In [1]:
!pip install basemap
!pip install dash
!pip install flask-ngrok

In [3]:
import os



In [4]:
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import plotly.io as pio
import numpy as np
from mpl_toolkits.basemap import Basemap
import dash
from dash import dcc, html, Input, Output, State, ALL
from flask_ngrok import run_with_ngrok

In [16]:
pip freeze > requirements.txt

In [8]:
raw_url = "https://raw.githubusercontent.com/prongselk/krillguard/main/KrillGUARD_public.xlsx"
data = pd.read_excel(raw_url, sheet_name="Raw_Data")
data = data.truncate(before=6)
data = data.dropna(how='all')

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [9]:
data = data.dropna(subset=['Lat'])

In [10]:
data['Species'] = data['Species'].fillna('Unknown')
data['Genus'] = data['Genus'].fillna('Unknown')

In [11]:
for column in data.columns:
       if data[column].dtype == 'object':
           if all(isinstance(x, str) for x in data[column] if pd.notna(x)):
               data[column] = data[column].str.strip()
           else:
               print(f"Column '{column}' contains non-string values. Skipping .str.strip().")

Column 'Cupboard ID' contains non-string values. Skipping .str.strip().
Column 'Station' contains non-string values. Skipping .str.strip().
Column 'Date' contains non-string values. Skipping .str.strip().
Column 'Lat' contains non-string values. Skipping .str.strip().
Column 'End Depth' contains non-string values. Skipping .str.strip().


In [12]:
fig = px.scatter_geo(data, lat='Lat', lon='Long',
                     hover_name='Station',
                     hover_data = ['Station', 'Date', 'Gear', 'Species'],
                     color = 'Expedition', opacity = 0.5,
                     color_discrete_sequence=px.colors.qualitative.Set2)

fig.update_layout(geo=dict(bgcolor='#e4f7fb'))
fig.show()
pio.write_html(fig, file='expeditions_map.html', auto_open=True)

In [13]:
def fix_species(row):
    if row['Species'] == "Unknown" and row['Genus'] != "Unknown":
        return f"{row['Genus']} sp."
    return row['Species']

data['Species'] = data.apply(fix_species, axis=1)

In [14]:
grouped_species = {}
for genus, group in data.groupby('Genus'):
    species_list = sorted(group['Species'].unique().tolist())
    grouped_species[genus] = species_list


non_unknown_genera = sorted([g for g in grouped_species.keys() if g != "Unknown"])
if "Unknown" in grouped_species:
    non_unknown_genera.append("Unknown")


species_checklist_layout = []
for genus in non_unknown_genera:
    species_checklist_layout.append(
        html.Details([
            html.Summary(genus, style={'cursor': 'pointer', 'fontSize': '14px','font-family': 'Helvetica'}),
            dcc.Checklist(
                id={'type': 'species-checklist', 'index': genus},
                options=[{'label': species, 'value': species} for species in grouped_species[genus]],

                value=grouped_species[genus],
                style={'fontSize': '12px', 'lineHeight': '1.5', 'fontStyle': 'italic'},
                inputStyle={"margin-right": "5px"}
            )
        ], open=False)
    )


In [15]:
app = dash.Dash(__name__)
run_with_ngrok(app.server)

app.layout = html.Div(style={'display': 'flex'}, children=[
    html.Div(style={'width': '20%', 'padding': '50px', 'backgroundColor': '#f8f8f8'}, children=[
        html.H4("Toggle species:", style={'fontSize': '14px', 'font-family': 'Helvetica'}),
        html.Button(
            "Deselect All",
            id="deselect-button",
            n_clicks=0,
            style={'marginBottom': '10px', 'fontSize': '12px'}
        ),
        html.Button(
            "Select All",
            id="select-all-button",
            n_clicks=0,
            style={'marginBottom': '10px', 'fontSize': '12px'}
        ),
        html.Div(species_checklist_layout),


    ]),

    html.Div(style={'width': '80%'}, children=[
        html.H1("Discovery Expeditions: Krill Station Data from the IOS Collection",
                style={'textAlign': 'center',
                       'fontSize':'16px',
                       'font-family':'Helvetica'}),
        html.H2("Click on legend to select expeditions, hover over points to see details",
               style={'textAlign':'right',
                      'fontSize':'12px',
                      'font-family':'Helvetica',
                      'fontWeight': 'normal',
                      'marginLeft': '20px',
                      'marginBottom': '5px'}),
        dcc.Graph(id='map-graph')
    ])
])

@app.callback(
    Output({'type': 'species-checklist', 'index': ALL}, 'value'),
    Input('deselect-button', 'n_clicks'),
    Input('select-all-button', 'n_clicks'),
    State({'type': 'species-checklist', 'index': ALL}, 'options')
)
def update_checklists(deselect_clicks, select_clicks, options_list):
    ctx = dash.callback_context
    if not ctx.triggered:

        return [[opt['value'] for opt in options] for options in options_list]

    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]
    if triggered_id == 'deselect-button':
        return [[] for _ in options_list]
    elif triggered_id == 'select-all-button':
        return [[opt['value'] for opt in options] for options in options_list]

    return dash.no_update


@app.callback(
    Output('map-graph', 'figure'),
    Input({'type': 'species-checklist', 'index': ALL}, 'value')
)
def update_map(list_of_values):

    selected_species = [species for sublist in list_of_values for species in sublist]
    filtered_data = data[data['Species'].isin(selected_species)]

    fig = px.scatter_geo(
        filtered_data,
        lat='Lat',
        lon='Long',
        hover_name='Station',
        hover_data=['Station', 'Date', 'Gear', 'Species'],
        color='Expedition',
        opacity=0.8,
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    fig.update_layout(geo=dict(bgcolor='#e4f7fb'))
    return fig

if __name__ == '__main__':
    port = int(os.environ.get("PORT", 8051))
    app.run_server(host='0.0.0.0', port=port)

<IPython.core.display.Javascript object>

In [23]:
!jupyter nbconvert --to script krillguard_map_app.ipynb


[NbConvertApp] WARNING | pattern 'krillguard_map_app.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--exec

In [24]:
!ls


expeditions_map.html  requirements.txt	sample_data
